In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/HIV_Resistance/GSE117748'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"MicroRNA-mediated suppression of the TGF-β pathway confers transmissible and reversible CDK4/6 inhibitor resistance"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell type: Immortalized cell line']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available
if 'gene' in '!Series_title' or 'gene' in '!Series_summary':
    is_gene_available = True

# Attempt to identify data availability for trait, age and gender
sample_characteristics = {
    0: ['cell type: Immortalized cell line']
}

# Mapping the data: Initially set to None as there's no provided keys in sample characteristics pointing to relevant data
trait_row = None
age_row = None
gender_row = None

# Define data type conversion functions
def convert_trait(value):
    val = value.split(':')[1].strip()  # Extract value after colon
    if val in ['resistant', 'nonresistant']:
        return 1 if val == 'resistant' else 0
    return None

def convert_age(value):
    val = value.split(':')[1].strip()  # Extract value after colon
    try:
        return float(val)
    except ValueError:
        return None

def convert_gender(value):
    val = value.split(':')[1].strip()  # Extract value after colon
    if val in ['male', 'female']:
        return 1 if val == 'male' else 0
    return None

# Save cohort information
save_cohort_info('GSE117748', './preprocessed/HIV_Resistance/cohort_info.json', is_gene_available, trait_row is not None)

# Since trait_row is None, skip Clinical Feature Extraction
